#Limpieza servicio logopedia
#### **NO hay datos de ALBACETE**


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.5 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [4]:
# # Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
# folder_path = '/content/drive/My Drive/PFG_FASPAS/SAAF'
# docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
# print(docs_xlsx)

# # Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
# folder_path_csv = '/content/drive/My Drive/PFG_FASPAS/Zonas_Prioritarias'
# docs_csv = [f for f in os.listdir(folder_path_csv) if f.endswith('.csv')]
# print(docs_csv)

In [5]:
# dic_dataframes = {}
# dic_zonas = {}

# for doc in docs_xlsx:
#     entire_path = os.path.join(folder_path, doc)
#     df = pd.read_excel(entire_path)
#     # Uso el nombre del archivo como clave
#     dic_dataframes[doc] = df

# for doc in docs_csv:
#   entire_path = os.path.join(folder_path_csv, doc)
#   df = pd.read_csv(entire_path)
#   # Uso el nombre del archivo como clave
#   dic_zonas[doc] = df

In [6]:
# # Imprimir todas las claves
# for clave in dic_dataframes.keys():
#     print(clave)

In [7]:
# CU_SAAF = dic_dataframes.get("CU_SAAF.xlsx")
# CU_AT = dic_dataframes.get("CU_AT.xlsx")
# CU_EMPLEO = dic_dataframes.get("CU_EMPLEO.xlsx")
# CU_LOG = dic_dataframes.get("CU_LOG.xlsx")
# # CU_SOCIOS = dic_dataframes.get("CU_SOCIOS.xlsx")

In [208]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
CU_LOG = pd.read_excel('/content/CU_LOG.xlsx')
TO_LOG = pd.read_excel('/content/TO_LOG.xlsx')
# AB_LOG = pd.read_excel('/content/AB_LOG.xlsx')
GU_LOG = pd.read_excel('/content/GU_LOG.xlsx')
CR_LOG = pd.read_excel('/content/CR_LOG.xlsx')

In [209]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA NACIMIENTO', 'Fecha nacimiento', 'Inicio Tratamiento', 'FECHA INICIO/FECHA REVISIÓN']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

def mapping_names(df):
    # Diccionario con los mapeos deseados
    columns_map = {
        'C. POSTAL': 'CP',
        'MOMENTO DE APARICION DE LA SORDERA': 'MOMENTO APARICION',
        'SIST. COMUNICACION': 'SISTEMA COMUNICACION',
        'SISTEMA DE COMUNICACION': 'SISTEMA COMUNICACION',
        'TIPO': 'TIPO SORDERA',
        'TIPO HIPOACUSIA': 'TIPO SORDERA',
        'GRADO': 'GRADO SORDERA',
        'TIPO PROTESIS': 'PROTESIS',
        'AUDIF/I.C': 'PROTESIS',
        'FECHA INICIO/FECHA REVISION': 'INICIO TRATAMIENTO'
    }

    # Crear un nuevo diccionario para los nombres de columnas
    rename_columns = {}

    # Iterar sobre las columnas y aplicar el mapeo
    for col in df.columns:
        norm_col = columns_map.get(col, col)
        rename_columns[col] = norm_col

    # Renombrar las columnas del DataFrame
    df.rename(columns=rename_columns, inplace=True)
    return df

def CP_year(df):
  if not pd.api.types.is_datetime64_any_dtype(df['FECHA NACIMIENTO']):
    df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')

  df['FECHA NACIMIENTO'] = df['FECHA NACIMIENTO'].apply(lambda x: np.nan if str(x).isdigit() or x == '(ADULTO)' else x)
  # df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')
  if 'CP' in df.columns:
    df['CP'] =df['CP'].astype('Int64')

    df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

    df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')

    CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()

    df = df[~CP_loc_filter]
  else:
    df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

    df = df.dropna(subset=['LOCALIDAD'], how='all')

    loc_filter = df['LOCALIDAD'].isnull()

    df = df[~loc_filter]
  return df

def norm_disc(val):
    if pd.isna(val):
        return val  # Retorna NaN tal como está
    if isinstance(val, str):  # Verificar si el valor es una cadena
        if val.startswith('SI'):
            if '%' in val:
                return int(val.split('(')[1].split('%')[0])  # Extraer el número del porcentaje
            return 'SI'  # Convertir "SI" y variantes sin número a "SI"
        if val.startswith('NO'):
            return 'NO'  # Convertir "NO TIENE" y "NO" a "NO"
        if 'PERM' in val:
            return int(val.split()[0])  # Extraer el número antes de "PERMANENTE"
        return val  # Retorna el valor tal cual si no se aplica ninguna condición anterior
    return val  # Maneja números directamente si no son cadenas

def norm_dep(value):
    if pd.isna(value):
        return value  # Retorna NaN tal como está
    if isinstance(value, str):
        value = value.strip()  # Limpiar espacios en blanco
        if value.upper() == 'NO' or value.upper() == 'NO TIENE' or value == 'N0':
            return 'NO'
        elif value.upper() == 'SI':
            return 'SI'
        elif 'SI TIPO' in value:
            return value.split('SI ')[1]  # Extraer solo la parte después de "SI "
        else:
            return value
    elif isinstance(value, int):
        if value in [1, 2, 3]:
            return f'TIPO {value}'
    return value

def other_columns(df):
  if 'PROTESIS' in df.columns:
    df['PROTESIS'] = df['PROTESIS'].fillna('NO TIENE')
    df['AUD O PROTESIS'] = np.where(df['PROTESIS'] == 'NO TIENE', 'NO', 'SI')
    df.drop('PROTESIS', axis=1, inplace=True)

  if 'TIPO SORDERA' in df.columns:
    mapeo_sordera = {
    'NEUROSENSORIAL': 'NEUROSENSORIAL',
    'NEUSENSORIAL': 'NEUROSENSORIAL',
    'HIPOACUSIA NEUROSENSORIAL': 'NEUROSENSORIAL',
    'HIPOACUSIA NEUROSENSORIAL PROFUNDA': 'NEUROSENSORIAL',
    'NEURO': 'NEUROSENSORIAL',
    'NEURO ??': 'NEUROSENSORIAL',

    'CONDUCTIVA': 'CONDUCTIVA',
    'HIPOACUSIA DE CONDUCCION': 'CONDUCTIVA',
    'HIPOACUSIA CONDUCTIVA UNILATERAL': 'CONDUCTIVA',
    'CONDUC': 'CONDUCTIVA',

    'MIXTA': 'MIXTA',
    'HIPOACUSIA MIXTA': 'MIXTA',
    'HIPOACUSIA MIXTA Y NEUROSENSORIAL': 'MIXTA',

    'HIPOACUSIA': 'HIPOACUSIA',
    'HIPOACUSIA ENDOCOCLEAR': 'HIPOACUSIA',

    'DIFICULTADES DEL LENGUAJE': 'DIFICULTADES DEL LENGUAJE',
    'RETRASO DEL LENGUAJE': 'RETRASO DEL LENGUAJE',

    '-': 'INDEFINIDO',
    'nan': 'INDEFINIDO'
    }
    df['TIPO SORDERA'] = df['TIPO SORDERA'].map(mapeo_sordera).fillna('INDEFINIDO')
  if 'GRADO DISCAPACIDAD' in df.columns:
    df['GRADO DISCAPACIDAD'] = df['GRADO DISCAPACIDAD'].apply(norm_disc)
  if 'LOCALIDAD' in df.columns:
    # Limpiar los datos eliminando contenido entre paréntesis
    df['LOCALIDAD'] = df['LOCALIDAD'].str.replace(r'\s*\([^)]*\)', '', regex=True)
  if 'MOMENTO APARICION' in df.columns:
    mapeo_apar = {
        'PRE': 'PRELOCUTIVA',
        'POST': 'POSTLOCUTIVA',
        'PERI': 'PERILOCUTIVA',
        '-': 'INDEFINIDO',
        'nan': 'INDEFINIDO'
    }
    df['MOMENTO APARICION'] = df['MOMENTO APARICION'].map(mapeo_apar).fillna('INDEFINIDO')
  if 'GRADO DEPENDENCIA' in df.columns:
    df['GRADO DEPENDENCIA'] = df['GRADO DEPENDENCIA'].apply(norm_dep)
  return df

def year_conversion(df):
  df['presente'] = 1

  # Crear pivot table
  pivot_df = df.pivot_table(index='FECHA NACIMIENTO', columns='ANO ATENCION', values='presente', fill_value=0, aggfunc='max')

  # Restablecer el índice para hacer que id_persona sea una columna otra vez
  pivot_df.reset_index(inplace=True)

  df_original_clean = df.drop_duplicates(subset=['FECHA NACIMIENTO'])
  df_final = pd.merge(df_original_clean, pivot_df, on=['FECHA NACIMIENTO'], how='left')

  return df_final

def convert_year_columns_to_int(df):
    # Crear un nuevo diccionario para mapear nombres antiguos a nuevos
    new_columns = {}
    for col in df.columns:
        # Intentar convertir la columna a entero si es posible
        try:
            # Convertir a entero si el nombre de la columna es numérico
            new_columns[col] = int(col)
        except ValueError:
            # Mantener el nombre original si no es un año
            new_columns[col] = col

    # Renombrar las columnas usando el diccionario mapeado
    df.rename(columns=new_columns, inplace=True)
    return df

def CU_year_conversion(df):
    # Asegurar que todos los valores en 'ANO ATENCION' sean strings
    df['ANO ATENCION'] = df['ANO ATENCION'].astype(str)

    # Normalizar los datos: eliminar espacios y caracteres extraños
    df['ANO ATENCION'] = df['ANO ATENCION'].apply(lambda x: x.replace(' ', '').replace('_', '').split('-'))

    # Explode de los años de atención para preparar para get_dummies
    df_exploded = df.explode('ANO ATENCION')

    # Aplicar get_dummies para convertir años en columnas binarias
    df_one_hot = pd.get_dummies(df_exploded['ANO ATENCION'].astype(str))

    # Limpiar los nombres de columnas eliminando espacios y caracteres no deseados
    df_one_hot.columns = df_one_hot.columns.str.replace('[^0-9]', '', regex=True).str.strip()

    # Consolidar columnas duplicadas resultantes de la limpieza (suma)
    df_one_hot = df_one_hot.groupby(df_one_hot.columns, axis=1).sum()

    # Agregar las otras columnas para hacer un merge posteriormente
    df_one_hot = pd.concat([df_exploded[['FECHA NACIMIENTO']], df_one_hot], axis=1)

    # Agrupar por FECHA NACIMIENTO y sumar (esto asegura que solo habrá 0s y 1s)
    df_final = df_one_hot.groupby(['FECHA NACIMIENTO']).sum().reset_index()

    # Merge el DataFrame original con el DataFrame final transformado
    df_merged = pd.merge(df, df_final, on=['FECHA NACIMIENTO'], how='left')
    df_merged = convert_year_columns_to_int(df_merged)

    return df_merged

def keep_columns(df):
    desired_columns = ['FECHA NACIMIENTO', 'GENERO', 'LOCALIDAD', 'CP', 'PROVINCIA','MOMENTO APARICION','TIPO SORDERA', 'GRADO PERDIDA', 'AUD O PROTESIS', 'GRADO DEPENDENCIA', 'GRADO DISCAPACIDAD', 'SISTEMA DE COMUNICACION',
                       'YEAR NACIMIENTO', 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
    # Filtrar el DataFrame para incluir solo las columnas que existen en el DataFrame
    filtered_df = df[df.columns.intersection(desired_columns)]
    return filtered_df

In [210]:
# # Suponiendo que 'dict_of_dfs' es tu diccionario de DataFrames
# for df_key, df in dic_dataframes.items():
#     dic_dataframes[df_key] = transform_df(df)
#     dic_dataframes[df_key] = mapping_names(df)
#     dic_dataframes[df_key] = kept_columns(df)
#     dic_dataframes[df_key] = CP_year(df)
#     dic_dataframes[df_key] = other_columns(df)
#     dic_dataframes[df_key] = delete_not_important_columns(df)
#     dic_dataframes[df_key] = left_columns_norm(df)

**OJO CR NO TIENE CP NI LOCALIDAD, NO PUEDO CRUZAR LOS DATOS CON NADA**

In [211]:
# CÓDIGO DE USO PARA EL CURRO
CU_LOG = transform_df(CU_LOG)
CU_LOG = mapping_names(CU_LOG)
CU_LOG = CP_year(CU_LOG)
CU_LOG = other_columns(CU_LOG)
CU_LOG = CU_year_conversion(CU_LOG)
CU_LOG = keep_columns(CU_LOG)

GU_LOG = transform_df(GU_LOG)
GU_LOG = mapping_names(GU_LOG)
GU_LOG = CP_year(GU_LOG)
GU_LOG = other_columns(GU_LOG)
GU_LOG = year_conversion(GU_LOG)
GU_LOG = keep_columns(GU_LOG)

TO_LOG = transform_df(TO_LOG)
TO_LOG = mapping_names(TO_LOG)
TO_LOG = CP_year(TO_LOG)
TO_LOG = other_columns(TO_LOG)
TO_LOG = year_conversion(TO_LOG)
TO_LOG = keep_columns(TO_LOG)

CR_LOG = transform_df(CR_LOG)
CR_LOG = mapping_names(CR_LOG)
# CR_LOG = CP_year(CR_LOG)
CR_LOG = other_columns(CR_LOG)
CR_LOG = year_conversion(CR_LOG)
CR_LOG = keep_columns(CR_LOG)

**POR QUÉ APARECEN AÑOS X E Y???**

In [235]:
CLM_LOG = pd.concat([CU_LOG, CR_LOG, GU_LOG, TO_LOG])

In [237]:
# Para bajarse los excel al ordena
!pip install openpyxl

In [239]:
ruta = r'\\windows.ocaso.pri\UsuariosCentral_01\AM5M\My Documents\Ana Alonso\PFG\DATO\LOG NORM\CLM_LOG_norm.xlsx'
# Exportar el DataFrame a un archivo Excel
CLM_LOG.to_excel(ruta, index=False, engine='openpyxl')

ruta = r'\\windows.ocaso.pri\UsuariosCentral_01\AM5M\My Documents\Ana Alonso\PFG\DATO\LOG NORM\CU_LOG_norm.xlsx'
CU_LOG.to_excel(ruta, index=False, engine='openpyxl')

ruta = r'\\windows.ocaso.pri\UsuariosCentral_01\AM5M\My Documents\Ana Alonso\PFG\DATO\LOG NORM\GU_LOG_norm.xlsx'
GU_LOG.to_excel(ruta, index=False, engine='openpyxl')

ruta = r'\\windows.ocaso.pri\UsuariosCentral_01\AM5M\My Documents\Ana Alonso\PFG\DATO\LOG NORM\TO_LOG_norm.xlsx'
TO_LOG.to_excel(ruta, index=False, engine='openpyxl')

# ruta = r'\\windows.ocaso.pri\UsuariosCentral_01\AM5M\My Documents\Ana Alonso\PFG\DATO\LOG NORM\AB_LOG_norm.xlsx'CR_LOG.to_excel(ruta, index=False, engine='openpyxl')
# AB_LOG.to_excel(ruta, index=False, engine='openpyxl')

In [242]:
from pathlib import Path

my_documents = Path.home() / 'Documentos'

# Ruta completa donde guardar el archivo
ruta_archivo = my_documents / 'Ana Alonso' / 'PFG' / 'datos_exportados.xlsx'

# Crear directorios si no existen
ruta_archivo.parent.mkdir(parents=True, exist_ok=True)

# Exportar a Excel
CLM_LOG.to_excel(ruta_archivo, index=False, engine='openpyxl')